# Aries Basic Controller Example - Alice
# DID Exchange - Inviter

In this notebook we'll be initiating the aries [DID Exchange](https://github.com/hyperledger/aries-rfcs/tree/master/features/0023-did-exchange) protocol using the aries_basic_controller package. 

This notebook has the following phases:

1. Pull in dependencies
2. Instantiate the controller for the aries agent (See the docker-compose.yml)
3. Set up a listener for basicmessages events emitted by the controller when it receives a webhook
4. Use the controller to create an invitation from our agent
5. Copy the invitation output from 4 and move over to Bob's [notebook](http://localhost:8889)

<b>Carry on in Bob's notebook</b>
    
12. Accept Request for Connection
13. Send Trust Ping to Bob

    



### 1. Pull in dependencies

In [1]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


### 2. Instatiate the controller for our Agent

The arguments depend on how the aca-py agent was initiated. See the manage and docker-compose.yml files for more details.

In [2]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, connections=True)

### 3. Set up a message listener running as a service on our controller

The default handlers are optional, they are used to manage state of protocol objects and print logs.

In [3]:
def messages_handler(payload):
    connection_id = payload["connection_id"]
    asyncio.get_event_loop().create_task(agent_controller.messaging.send_message(connection_id, "This is a response from Alice"))
    print("Handle message", payload, connection_id)


message_listener = {
    "handler": messages_handler,
    "topic": "basicmessages"
}

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners([message_listener], defaults=True)

### 4. Use the controller to create an invitation from our agent


In [4]:
# Create Invitation
invite = await agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
print("Connection ID", connection_id)
print("Invitation")
print(invite)

Connection ID 5d465532-247d-47ce-a56f-c4ef65e7a5b9
Invitation
{'connection_id': '5d465532-247d-47ce-a56f-c4ef65e7a5b9', 'invitation': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '67124ffd-43f4-4161-9926-ec815f1376e3', 'recipientKeys': ['9ywxqPKtayXpyiQ7K8YedLwJq5osGpJ6HUELvdCv4hdr'], 'serviceEndpoint': 'http://172.17.0.1:8020', 'label': 'Alice'}, 'invitation_url': 'http://172.17.0.1:8020?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiNjcxMjRmZmQtNDNmNC00MTYxLTk5MjYtZWM4MTVmMTM3NmUzIiwgInJlY2lwaWVudEtleXMiOiBbIjl5d3hxUEt0YXlYcHlpUTdLOFllZEx3SnE1b3NHcEo2SFVFTHZkQ3Y0aGRyIl0sICJzZXJ2aWNlRW5kcG9pbnQiOiAiaHR0cDovLzE3Mi4xNy4wLjE6ODAyMCIsICJsYWJlbCI6ICJBbGljZSJ9'}


### 5. Copy the invitation output from 4 and move over to the Bob notebook (localhost:8889) 

### 11. Accept Request for Connection

In [5]:
# Accept Request for Invite created
connection = await agent_controller.connections.accept_request(connection_id)
print("ACCEPT REQUEST")
print(connection)
print("state", connection["state"])

True
ACCEPT REQUEST
{'updated_at': '2020-07-03 11:44:22.782370Z', 'invitation_mode': 'once', 'routing_state': 'none', 'invitation_key': '9ywxqPKtayXpyiQ7K8YedLwJq5osGpJ6HUELvdCv4hdr', 'their_did': 'WMt1wVYjkRNGsWt2jaXvLS', 'state': 'response', 'accept': 'manual', 'initiator': 'self', 'my_did': 'G4c8DyQsdbBWnQZfyPtXct', 'their_label': 'Bob', 'created_at': '2020-07-03 11:43:56.161836Z', 'connection_id': '5d465532-247d-47ce-a56f-c4ef65e7a5b9'}
state response


### 12. Send Trust Ping to activate the conneciton

In [6]:
trust_ping = await agent_controller.messaging.trust_ping(connection_id, "hello")
print("Trust Ping", trust_ping)

Trust Ping {'thread_id': '67c9c190-f58e-4a59-91c4-a09b84345e0b'}


####  12.1 Check if connection is active

In [7]:
connection = await agent_controller.connections.get_connection(connection_id)
print(connection)
print("Is Active?", connection["state"])

{'updated_at': '2020-07-03 11:44:28.525278Z', 'invitation_mode': 'once', 'routing_state': 'none', 'invitation_key': '9ywxqPKtayXpyiQ7K8YedLwJq5osGpJ6HUELvdCv4hdr', 'their_did': 'WMt1wVYjkRNGsWt2jaXvLS', 'state': 'active', 'accept': 'manual', 'initiator': 'self', 'my_did': 'G4c8DyQsdbBWnQZfyPtXct', 'their_label': 'Bob', 'created_at': '2020-07-03 11:43:56.161836Z', 'connection_id': '5d465532-247d-47ce-a56f-c4ef65e7a5b9'}
Is Active? active


###  13 Send a basic message over DIDComm channel

See [aries-rfc](https://github.com/hyperledger/aries-rfcs/tree/master/features/0095-basic-message)

In [8]:
message = await agent_controller.messaging.send_message(connection_id,"hello from Alice")
print("BASIC MESSAGE - Alice -> Bob")
print(message)

BASIC MESSAGE - Alice -> Bob
{}


Error handling request
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/aiohttp/web_protocol.py", line 418, in start
    resp = await task
  File "/opt/conda/lib/python3.7/site-packages/aiohttp/web_app.py", line 458, in _handle
    resp = await handler(request)
  File "/aries_basic_controller/aries_controller.py", line 59, in _receive_webhook
    await self.handle_webhook(topic, payload)
  File "/aries_basic_controller/aries_controller.py", line 65, in handle_webhook
    pub.sendMessage(topic, payload=payload)
  File "/opt/conda/lib/python3.7/site-packages/pubsub/core/publisher.py", line 216, in sendMessage
    topicObj.publish(**msgData)
  File "/opt/conda/lib/python3.7/site-packages/pubsub/core/topicobj.py", line 452, in publish
    self.__sendMessage(msgData, topicObj, msgDataSubset)
  File "/opt/conda/lib/python3.7/site-packages/pubsub/core/topicobj.py", line 482, in __sendMessage
    listener(data, self, allData)
  File "/opt/conda/lib/python3.7/si

## Terminate Controller & Stop Webhook Server

**Note: You will need to run this command when combining this example with others such as Issuer**

In [8]:
response = await agent_controller.terminate()
print(response)

None
